In [1]:
import requests
import base64
import time
import random

# --- Cấu hình ---
API_URL = "https://api.thucchien.ai/v1/images/generations"
API_KEY = "sk-hCLQr028vJYPIaU1j6h1zQ"
PROMPT = """
Black and white comic style photo, a Vietnamese police officer with a deceitful face was arrested by a calm, upright-faced Vietnamese policeman"""
NUM_IMAGES = 1  # Số lượng ảnh cần tạo

# --- Cấu hình retry ---
MAX_RETRIES = 10  # Số lần thử tối đa
BASE_DELAY = 1  # Thời gian chờ cơ bản (giây)
MAX_DELAY = 60  # Thời gian chờ tối đa (giây)

def make_api_call_with_retry():
    """
    Gọi API với cơ chế retry cho đến khi thành công
    """
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    
    payload = {
        "model": "imagen-4",
        "prompt": PROMPT,
        "n": NUM_IMAGES,
    }
    
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            print(f"🔄 Attempt {attempt}/{MAX_RETRIES} - Calling API...")
            
            response = requests.post(API_URL, headers=headers, json=payload, timeout=30)
            
            # Kiểm tra status code
            if response.status_code == 200:
                data = response.json()
                images = data.get("data", [])
                
                # Kiểm tra xem có ảnh được tạo không
                if images and len(images) > 0:
                    print(f"✅ Success! Generated {len(images)} images on attempt {attempt}")
                    return response, data
                else:
                    print(f"⚠️ Attempt {attempt}: No images in response. Retrying...")
                    print(f"Response data: {data}")
            else:
                print(f"❌ Attempt {attempt}: HTTP {response.status_code}")
                print(f"Error: {response.text}")
                
        except requests.exceptions.Timeout:
            print(f"⏰ Attempt {attempt}: Request timeout. Retrying...")
        except requests.exceptions.ConnectionError:
            print(f"🔌 Attempt {attempt}: Connection error. Retrying...")
        except requests.exceptions.RequestException as e:
            print(f"🚫 Attempt {attempt}: Request error: {e}. Retrying...")
        except Exception as e:
            print(f"💥 Attempt {attempt}: Unexpected error: {e}. Retrying...")
        
        # Tính toán thời gian chờ với exponential backoff + jitter
        if attempt < MAX_RETRIES:
            delay = min(BASE_DELAY * (2 ** (attempt - 1)) + random.uniform(0, 1), MAX_DELAY)
            print(f"⏳ Waiting {delay:.1f} seconds before next attempt...")
            time.sleep(delay)
    
    # Nếu đến đây thì đã hết số lần thử
    raise Exception(f"❌ Failed to get successful response after {MAX_RETRIES} attempts")

# --- Thực hiện gọi API với retry ---

In [4]:
try:
    response, data = make_api_call_with_retry()

    # --- Xử lý phản hồi thành công ---
    images = data.get("data", [])

    if images:
        for i, img_obj in enumerate(images, start=1):
            image_data = img_obj.get("b64_json")
            if image_data:
                image_bytes = base64.b64decode(image_data)
                file_name = f"generated_image_{i}.png"
                with open(file_name, "wb") as f:
                    f.write(image_bytes)
                print(f"✅ Image {i} saved to {file_name}")
            else:
                print(f"⚠️ Image {i} has no base64 data.")
    else:
        print("❌ No images generated. Response data:")
        print(data)

except Exception as e:
    print(f"💥 Final error: {e}")
    print("🔄 You can run this cell again to retry the entire process.")

🔄 Attempt 1/10 - Calling API...
✅ Success! Generated 1 images on attempt 1
✅ Image 1 saved to generated_image_1.png
